In [1]:
import pandas as pd 

In [3]:
df1 = pd.read_csv('Data06.csv')
df1.head(2)

,회원번호,조합원상태,주소-구,주소-동,성별,연령,연령대,총구매금액,총구매수량,1회방문시구매금액(평균),배송서비스신청여부,모바일알람여부,Gold_member
0,272369856,정상회원,수지구,풍덕천동,여,45.0,40대,5733884,546.5,47782,미신청,수신,VIP
1,1506656256,정상회원,수지구,풍덕천동,여,36.0,30대이하,673414,90.0,35443,미신청,.,normal


# 1. 데이터 준비 및 가공

- Y (Target) : **Gold_member** / X (Feature) ... 
- Feature Selection 
- Missing Value 

In [5]:
df2 = df1.drop(columns=['회원번호','주소-동','연령대','1회방문시구매금액(평균)'])
df2.head()

,조합원상태,주소-구,성별,연령,총구매금액,총구매수량,배송서비스신청여부,모바일알람여부,Gold_member
0,정상회원,수지구,여,45.0,5733884,546.5,미신청,수신,VIP
1,정상회원,수지구,여,36.0,673414,90.0,미신청,.,normal
2,정상회원,수지구,여,34.0,655919,66.0,미신청,.,normal
3,정상회원,수지구,여,51.0,2984534,252.1,미신청,.,normal
4,정상회원,수지구,여,51.0,1901488,152.0,신청,.,normal


In [7]:
df2.isnull().sum() # 결측값 확인 

조합원상태          0
주소-구           0
성별             1
연령             1
총구매금액          0
총구매수량          0
배송서비스신청여부      0
모바일알람여부        0
Gold_member    0
dtype: int64

In [9]:
df3 = df2.dropna() # 결측값 제거 
df3 

,조합원상태,주소-구,성별,연령,총구매금액,총구매수량,배송서비스신청여부,모바일알람여부,Gold_member
0,정상회원,수지구,여,45.0,5733884,546.5,미신청,수신,VIP
1,정상회원,수지구,여,36.0,673414,90.0,미신청,.,normal
2,정상회원,수지구,여,34.0,655919,66.0,미신청,.,normal
3,정상회원,수지구,여,51.0,2984534,252.1,미신청,.,normal
4,정상회원,수지구,여,51.0,1901488,152.0,신청,.,normal
...,...,...,...,...,...,...,...,...,...
6514,정상회원,수지구,남,32.0,52646,1.1,미신청,.,normal
6515,정상회원,분당구,여,46.0,61740,9.0,미신청,.,normal
6516,정상회원,기타,여,82.0,15507,2.0,미신청,.,normal
6517,정상회원,분당구,여,55.0,36374,2.0,미신청,.,normal


In [11]:
df3['Gold_member'].value_counts()

normal    6421
VIP         96
Name: Gold_member, dtype: int64

In [13]:
df3['Target'] = df3['Gold_member'].replace({'VIP':1, 'normal':0})

<ipython-input-13-75d24b0a67bc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Target'] = df3['Gold_member'].replace({'VIP':1, 'normal':0})


In [18]:
X = pd.get_dummies(df3.drop(columns=['Gold_member','Target'])) # One-hot Encoding 
Y = df3['Target']

# 2. 데이터 전처리( 분할 )

- sklearn 
    - model_selection
    - metrics
    - tree 

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

In [24]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3, 
                                                    random_state=2020)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(4561, 30)
(1956, 30)
(4561,)
(1956,)


# 3. 학습 

In [25]:
model = DecisionTreeClassifier()
model.fit(X_train,Y_train)

DecisionTreeClassifier()

# 4. 평가

In [26]:
Y_train_pred = model.predict(X_train) # 알고리즘이 학습을 얼마나 잘 시켰는가? 
Y_test_pred = model.predict(X_test) # 새로운 데이터가 들어올 때, 얼마나 좋은 성능을 나타내는가?

In [27]:
print(classification_report(Y_train,Y_train_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4495
           1       1.00      1.00      1.00        66

    accuracy                           1.00      4561
   macro avg       1.00      1.00      1.00      4561
weighted avg       1.00      1.00      1.00      4561



In [28]:
print(classification_report(Y_test,Y_test_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1926
           1       0.56      0.50      0.53        30

    accuracy                           0.99      1956
   macro avg       0.77      0.75      0.76      1956
weighted avg       0.99      0.99      0.99      1956

